In [39]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate,ChatPromptTemplate
from langchain.callbacks import StdOutCallbackHandler,StreamingStdOutCallbackHandler

chat = ChatOpenAI(model="gpt-4.1-nano", temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])


chef_template = ChatPromptTemplate.from_messages([
    ("system","You are a world class chef. You are given a list of ingredients easy to find in a kitchen and you need to create a recipe for a dish easy to make. and you love meat and."),
    ("human","I want to cook {cousine}food."),
])

chef_chain = chef_template | chat




In [40]:
chef_template = ChatPromptTemplate.from_messages([
    ("system","You are a world class chef. You are given a list of ingredients easy to find in a kitchen and you need to create a recipe for a dish easy to make."),
    ("human","I want to cook {cousine}food."),
])

chef_chain = chef_template | chat



In [41]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a vegetarian chef. specialized in traditional recepis vegeterian. you find alternatives for non-vegetarian ingredients. you don't radically modify the recipe. if there is alternative for a food just say you don't know how to replace it"),
    ("human","{recipe}"),
])

veg_chef_chain = veg_chef_prompt | chat


final_chain = {"recipe":chef_chain} | veg_chef_chain


final_chain.invoke({"cousine":"Japanese"})








Great choice! Here's a simple and delicious Japanese-inspired dish you can make with common kitchen ingredients: **Teriyaki Chicken Bowl**. It's easy to prepare and full of flavor.

### Ingredients:
- 2 chicken breasts or thighs
- 1/4 cup soy sauce
- 2 tablespoons honey or sugar
- 1 tablespoon rice vinegar (or white vinegar if unavailable)
- 1 tablespoon vegetable oil
- 2 cloves garlic, minced (or garlic powder)
- 1 teaspoon grated ginger (or ground ginger)
- Cooked rice (white or brown)
- Optional toppings: sliced green onions, sesame seeds, steamed vegetables (like broccoli or carrots)

### Instructions:

1. **Prepare the Teriyaki Sauce:**
   - In a small bowl, mix soy sauce, honey (or sugar), rice vinegar, minced garlic, and grated ginger. Set aside.

2. **Cook the Chicken:**
   - Heat vegetable oil in a skillet over medium heat.
   - Season chicken with a little salt and pepper if desired.
   - Add chicken to the skillet and cook until browned and cooked through (about 6-8 minutes 

AIMessageChunk(content="Certainly! To make this dish vegetarian, you can replace the chicken with a plant-based protein. Some good options include:\n\n- Tofu (firm or extra firm, pressed and sliced)\n- Tempeh (sliced into strips)\n- Seitan (if available)\n- Mushrooms (like portobello or shiitake, sliced)\n\nHere's how you can adapt the recipe:\n\n### Vegetarian Teriyaki Bowl\n\n**Replace the chicken with:**  \n- 1 block of firm tofu, pressed and sliced into strips, or  \n- 1 cup of sliced tempeh or mushrooms\n\n**Cooking instructions:**  \n- For tofu or tempeh: Sauté in a little oil until golden on all sides before adding the sauce.  \n- For mushrooms: Sauté until tender and browned.\n\nThe rest of the recipe remains the same. The soy-based teriyaki sauce pairs beautifully with these alternatives, creating a delicious vegetarian dish without radically modifying the original recipe. Enjoy!")

In [36]:

from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        items = text.strip().split(",")
        return list(map(str.strip, items))


comma_parser = CommaOutputParser()
comma_parser.parse("Hello,how ,are,you,!")
template = ChatPromptTemplate.from_messages([
   ("system", "You are a list generating machine. Everything you area aksed will be answerd with a comma separated list of max {max_items} in lowercase. Do not reply with antyhing else"),
    
    ("human", "{question}"),
])

prompt = template.format_messages(max_items=10, question="What are the colors?")
response = chat.predict_messages(prompt)

comma_parser = CommaOutputParser()
comma_parser.parse(response.content)

red,blue,green,yellow,black,white,orange,purple,pink,brown

['red',
 'blue',
 'green',
 'yellow',
 'black',
 'white',
 'orange',
 'purple',
 'pink',
 'brown']

In [37]:
chain = template | chat | comma_parser

chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

bulbasaur, ivysaur, venusaur, charmander, charmeleon

['bulbasaur', 'ivysaur', 'venusaur', 'charmander', 'charmeleon']

In [38]:


chat = ChatOpenAI(model="gpt-4.1-nano", temperature=0.1)
template = PromptTemplate.from_template("What is the distance between {country1} and {country2}?")
prompt = template.format(country1="Mexico", country2="Thailand")


template = ChatPromptTemplate.from_messages([
   ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "안녕 내 이름은 {name}이야!"),
    ("human", "Hello, What is your name? And what is the distance between {country1} and {country2}? "),
])

prompt = template.format_messages(language="japanese", name="Socrates", country1="Mexico", country2="Thailand")

chat.predict_messages(prompt)

AIMessage(content='こんにちは、私は地理の専門家です。私の名前はSocratesです。メキシコとタイの距離は直線距離で約13,000キロメートルです。')